In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from torch.utils.data import DataLoader

from utils import get_data_matrix
from datasets import MEGDataset


In [2]:
dataset = MEGDataset("../data_2/Intra/train")
dataloader = DataLoader(dataset, batch_size=16)


In [6]:
dataset[25]

(array([[0.9043036 , 0.90431225, 0.9031673 , ..., 0.9051233 , 0.9053185 ,
         0.9053835 ],
        [0.90112513, 0.899738  , 0.8992395 , ..., 0.8985936 , 0.8983552 ,
         0.89831185],
        [0.91160387, 0.9126796 , 0.9130322 , ..., 0.9137009 , 0.91326475,
         0.91313756],
        ...,
        [0.8999317 , 0.90074193, 0.90079755, ..., 0.8916038 , 0.89170116,
         0.8923062 ],
        [0.90890497, 0.90976995, 0.90935874, ..., 0.9075012 , 0.9077777 ,
         0.90839803],
        [0.8896264 , 0.88880366, 0.8882635 , ..., 0.88805604, 0.8871869 ,
         0.8867576 ]], shape=(248, 248), dtype=float32),
 3)

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SimpleCNN(nn.Module):
    def __init__(self, num_classes=10):
        super(SimpleCNN, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Po 2x pooling rozmiar spada z 248 -> 124 -> 62
        self.fc1 = nn.Linear(32 * 62 * 62, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        # Input shape: [B, 1, 248, 248]
        x = self.pool(F.relu(self.conv1(x)))  # -> [B, 16, 124, 124]
        x = self.pool(F.relu(self.conv2(x)))  # -> [B, 32, 62, 62]
        x = x.view(x.size(0), -1)             # flatten -> [B, 32*62*62]
        x = F.relu(self.fc1(x))               # -> [B, 128]
        x = self.fc2(x)                       # -> [B, num_classes]
        return x


In [6]:
next(iter(dataloader)).shape

torch.Size([16, 248, 248])

In [5]:
model = SimpleCNN(num_classes=5)  # np. 5 klas
output = model(next(iter(dataloader)))
print(output.shape)  # [8, 5]


RuntimeError: Given groups=1, weight of size [16, 1, 3, 3], expected input[1, 16, 248, 248] to have 1 channels, but got 16 channels instead